# Advanced Querying Mongo

Importing libraries and setting up connection

In [ ]:
!pip install pymongo

In [1]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [2]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [4]:
db = client.get_database("ironhack")

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [6]:
db.list_collection_names()

['companies']

In [7]:
c = db.get_collection("companies")
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [89]:
proyec = {'name':1, '_id':0}
query = {'name':'Babelgum'}
Bg = list(c.find(query,proyec))
Bg

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [90]:
query = {"number_of_employees": {"$gt": 5000}}
comp_5000 = list(c.find(query).sort("number_of_employees", -1).limit(20))
#comp_5000

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [91]:
c1 = {"founded_year": {"$gt": 1999}}
c2 = {"founded_year": {"$lt": 2006}}
proyec = {"name":1, "_id":0, "founded_year":1}
query = {'$and':[c1, c2]}
comp_05 = list(c.find(query,proyec))
#comp_05

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [92]:
c1={'ipo.valuation_amount':{'$gt':100000000}}
c2 = {'founded_year':{'$lt':2010}}
proyec = {"name":1, "_id":0, "ipo":1}
query = {'$and':[c1, c2]}
comp_1010 = list(c.find(query,proyec))
#comp_1010

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [94]:
c1={'number_of_employees': {'$lt' : 1000}}
c2 = {'founded_year':{'$lt':2005}}
proyec = {'name':1, 'ipo':1, '_id':0}
query = {'$and':[c1, c2]}
list(c.find(query,proyec).sort('number_of_employees',-1).limit(10))

[{'name': 'Infinera Corporation',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 2007,
   'pub_month': 6,
   'pub_day': 7,
   'stock_symbol': 'INFN'}},
 {'name': 'NorthPoint Communications Group', 'ipo': None},
 {'name': '888 Holdings', 'ipo': None},
 {'name': 'Forrester Research',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'JPY',
   'pub_year': 1987,
   'pub_month': 6,
   'pub_day': 16,
   'stock_symbol': 'FORR'}},
 {'name': 'Webmetrics', 'ipo': None},
 {'name': 'SonicWALL', 'ipo': None},
 {'name': 'Cornerstone OnDemand',
  'ipo': {'valuation_amount': 900000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 5,
   'stock_symbol': 'CSOD'}},
 {'name': 'Buongiorno',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 2003,
   'pub_month': 7,
   'pub_day': 17,
   'stock_symbol': 'BNG'}},
 {'name': 'Cvent',
  'ipo': {'valuation_amount': None,
   'v

### 6. All the companies that don't include the `partners` field.

In [95]:
c1 = {"partners": {"$exists": False}}
comp_part = list(c.find(c1))
comp_part

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [96]:
c1 = {"category_code": {"$exists": False}}
comp_cat = list(c.find(c1))
comp_cat

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [97]:
c1 = {"number_of_employees": {"$lt": 1000}}
c2 = {"number_of_employees": {"$gt": 99}}
proyec = {"name": 1, "number_of_employees": 1, "_id": 0}
query = {"$and": [c1,c2]}
comp_100 = list(c.find(query,proyec))
#comp_100

### 9. Order all the companies by their IPO price in a descending order.

In [98]:
comp_ord = list(c.find().sort('ipo.valuation_amount',-1))
#comp_ord

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [99]:
proyec = {"name": 1, "number_of_employees": 1, '_id': 0}
c1 = {"number_of_employees": {"$exists": True}}
list(c.find(c1,proyec).limit(10).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [100]:
proyec = {"name": 1, "founded_year": 1, 'founded_month':1, '_id': 0}
list(c.find({"founded_month": {"$in": [7,8,9,10,11,12]}}, proyec).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005, 'founded_month': 10},
 {'name': 'Zoho', 'founded_year': 2005, 'founded_month': 9},
 {'name': 'Digg', 'founded_year': 2004, 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_year': 2005, 'founded_month': 11},
 {'name': 'eBay', 'founded_year': 1995, 'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [101]:
proyec = {"acquisition": 1, "name": 1, "founded_year": 1, '_id': 0}
c1 = {"founded_year":{"$lt": 2000}}
c2 = {"acquisition.price_amount": {"$gt": 10000000}}
query = {"$and": [c1,c2]}
comp_foun = list(c.find(query,proyec).sort("acquisition.price_amount",-1))
#comp_foun

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [102]:
proyec = {"name": 1, 'acquisition':1, '_id': 0}
c1 = {'acquisition.acquired_year':{'$gt': 2010}}
comp_acq = list(c.find(c1, proyec).sort('acquisition.price_amount',-1))
#comp_acq

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [103]:
proyec = {"name": 1, 'founded_year':1, '_id': 0}
c1 = {"founded_year": {"$exists": True}}
comp_fy = list(c.find(c1,proyec).sort('founded_year',-1))
#comp_fy

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [104]:
proyec = {"name": 1, 'founded_day':1, '_id': 0}
c1 = {'founded_day':{'$lte': 7}}
list(c.find(c1,proyec).sort('acquisition.price_amount',1).limit(10))

[{'name': 'Facebook', 'founded_day': 1},
 {'name': 'Joost', 'founded_day': 1},
 {'name': 'Babelgum', 'founded_day': 1},
 {'name': 'Gizmoz', 'founded_day': 1},
 {'name': 'eBay', 'founded_day': 1},
 {'name': 'Technorati', 'founded_day': 1},
 {'name': 'Yahoo!', 'founded_day': 1},
 {'name': 'Fox Interactive Media', 'founded_day': 1},
 {'name': 'Omnidrive', 'founded_day': 1},
 {'name': 'Geni', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [105]:
proyec = {"name": 1, "number_of_employees": 1, 'category_code':1, '_id': 0}
c1 = {'category_code':'web'}
c2= {'number_of_employees': {'$gt' : 4000}}
query = {'$and':[c1,c2]}
list(c.find(query,proyec).sort('number_of_employees'))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [106]:
proyec = {"name": 1, 'acquisition':1, '_id': 0}
c1 = {'acquisition.price_currency_code':'EUR'}
c2 = {'acquisition.price_amount':{'$gt':10000000}}
query = {'$and':[c1,c2]}
comp_eur = list(c.find(query,proyec))
#comp_eur

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [109]:
proyec = {"name": 1, "acquisition": 1, '_id': 0}
comp_1trs = list(c.find({"acquisition.acquired_month": {"$in": [1,2,3]}}, proyec).limit(10))
#comp_1trs

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [110]:
proyec = {"name":1, "acquisition.acquired_year":1, "_id":0}
c1 = {'founded_year': {'$gte':2000}}
c2 = {'founded_year': {'$lt':2010}}
c3 = {'acquisition.acquired_year': {'$nin':list(range(0,2012))}}
query = {'$and':[c1,c2,c3]}
comp_bet = list(c.find(query,proyec))
#comp_bet

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code